In [12]:
import os
import sys

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.subplots as subplots

from typing import Union, Literal

sys.path.append(os.path.abspath('..'))
from graphmodex import plotlymodex

In [13]:
fig = plotlymodex.frequency(df=df, x='x', covariate='value', colors=['black', 'red'], opacity=0.4)
fig

In [30]:
def subplot(figs:list, rows:int=1, cols:int=2, subplot_titles:list[str]=None,
            shared_xaxes:bool=False, shared_yaxes:bool=False,
            horizontal_spacing:float=0.1, vertical_spacing:float=0.1,
            specs:list=None, x_title:str='x', y_title:str='y',
            layout_kwargs:dict=None, subplots_kwargs:dict=None):

    fig = subplots.make_subplots(
        rows=rows, cols=cols, subplot_titles=subplot_titles, specs=specs, 
        shared_xaxes=shared_xaxes, shared_yaxes=shared_yaxes, 
        horizontal_spacing=horizontal_spacing, vertical_spacing=vertical_spacing,
        **(subplots_kwargs or {})
    )

    occupied = [[False for _ in range(cols)] for _ in range(rows)]

    def find_next_available():
        for r in range(rows):
            for c in range(cols):
                if not occupied[r][c]:
                    return r + 1, c + 1  # Plotly uses 1-based indexing
        raise ValueError("No available space for subplot.")

    def mark_occupied(r, c, rowspan=1, colspan=1):
        for i in range(rowspan):
            for j in range(colspan):
                if r + i < rows and c + j < cols:
                    occupied[r + i][c + j] = True

    for i, f in enumerate(figs):
        if specs:
            # Determine from specs if this subplot spans multiple rows/cols
            for r in range(rows):
                for c in range(cols):
                    spec = specs[r][c] if specs[r][c] is not None else {}
                    if not occupied[r][c]:
                        rowspan = spec.get('rowspan', 1)
                        colspan = spec.get('colspan', 1)
                        mark_occupied(r, c, rowspan, colspan)
                        row, col = r + 1, c + 1
                        break
                else:
                    continue
                break
        else:
            row, col = find_next_available()
            mark_occupied(row - 1, col - 1)

        # Add all traces from each input figure into subplot
        for trace in f.data:
            fig.add_trace(trace, row=row, col=col)

    # Set axis titles
    fig.update_xaxes(title_text=x_title)
    fig.update_yaxes(title_text=y_title)

    # Apply custom layout if any
    if layout_kwargs:
        fig.update_layout(**layout_kwargs)

    plotlymodex.main_layout(fig, width=1400, x=None, y=None)

    return fig

subplot(figs=[fig, fig])